<a href="https://colab.research.google.com/github/TirendazAcademy/Hugging-Face-Tutorials/blob/main/ChatBot-Zephyr-3B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U gradio transformers accelerate

In [2]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'stabilityai/stablelm-zephyr-3b')

In [5]:
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    'stabilityai/stablelm-zephyr-3b',
    trust_remote_code=True,
    device_map="auto")

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

In [10]:
class ChatBot:
  def __init__(self):
    self.history = []
  def predict(self, user_input,
              system_prompt="You are an expert analyst and provide assessment:"):
    prompt = [{'role': 'user', 'content': user_input + "\n" + system_prompt + ":"}]
    inputs = tokenizer.apply_chat_template(
        prompt,
        add_generation_prompt=True,
        return_tensors='pt',
    )
    tokens = model.generate(
        inputs.to(model.device),
        max_new_tokens=250,
        temperature=0.8,
        do_sample=False
    )
    response_text = tokenizer.decode(
        tokens[0],
        skip_special_tokens=False)
    del tokens
    torch.cuda.empty_cache()
    return response_text

In [11]:
bot = ChatBot()

In [12]:
import gradio as gr

In [15]:
examples = [["What is the dropout technique in machine learning?", "I want you to act as a machine learning engineer. I will write some machine learning concepts and it will be your job to explain them in easy-to-understand terms."]]

iface = gr.Interface(
    fn=bot.predict,
    title="An App with Zephyr-3b",
    examples=examples,
    inputs=["text", "text"],
    outputs="text",
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1a9dabbbbe37f02636.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
